## [5-2]교차 검증과 그리드 서치

- 핵심 키워드
  - 검증 세트
  - 교차 검증
  - 그리드 서치
  - 랜덤 서치

## 검증 세트(validation set)
: 테스트 세트를 사용하지 않으면 과대적합/과소적합 판단 어려움.  
-> 훈련세트를 또 나눠서 검증 세트(validation set)

#보통 20-30% 테스트와 검증 세트로 떼어놓음

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_test_split() 이용해서 훈련 세트와 테스트 세트 split!!

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

train_input과 train_target을 이용해서 다시.  
sub_input, sub_target과 검증세트 val_input, val_target 만듬

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [7]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증(cross validation)
> 교차 검증을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터 사용 가능


#훈련 세트를 세부분으로 나눠서 수행하는 것 -> 3-폴드 교차 검증  
통칭 k-폴드 교차 검증(k-fold cross validation)


#보통 5-fold cross validation or 10-fold cross validation 많이 사용

사이킷런에 cross_validate() 함수 존재  
(전신인 cross_val_score() 함수도 존재)

fit_time, score_time, test_score 키를 가진 딕셔너리 반환

cross_validate(cv=5) default : 5-fold cross validation

In [8]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00768113, 0.006881  , 0.00670505, 0.00590205, 0.00520992]), 'score_time': array([0.00054193, 0.00055909, 0.00047588, 0.00047112, 0.00043583]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [9]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


주의점: cross_validate() 훈련 세트를  섞어 폴드를 나누지 않음   
앞서 train_test_split() 함수로  전제 데이터를 섞은 후 훈련 세트를 준비했기 대문에 따로 섞을 필요 없었음

만약 교차 검증 할때 훈련 세트를 섞으러면 분할기(splitter) 지정

타깃 클래스를 골고루 나누기 위해 StatifiedKFold 사용

In [10]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝
: 사용자가 지정해야만 하는 파라미터를 하이퍼파라미터

>사람의 개입 없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술 'AutoML'이라고 부름

사이킷런에서 제공하는 그리드 서치(Grid Search)

GridSearchCV 클래스 하이퍼파라미터 탐색과 교차 검증 한번에 수행

In [12]:
from sklearn.model_selection import GridSearchCV
#Make var params
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [15]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

#njobs CPU 병렬 실행 여부       default:1   사용 x

In [16]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

가장 높은 점수의 모델(같은 매개변수로 다시 학습)  
best_estimator_ 속성에 저장

In [17]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


최적의 매개변수 best_params_ 속성에 저장

In [18]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_result_ 속성의 'mean_test_score' 키에 저장

In [19]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


numpy의 argmax() 함수 사용하면 가장 큰 값의 인덱스 값 리턴

In [20]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


numpy 모듈의 arange 함수는 반열린구간 [start, stop) 에서 step 의 크기만큼.  
일정하게 떨어져 있는 숫자들을 array 형태로 반환해 주는 함수

In [21]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [22]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [23]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [24]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치(Random Search)
: 랜덤 서치는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달

In [25]:
#수학 라이브러리
from scipy.stats import uniform, randint

In [28]:
rgen = randint(0, 10)
rgen.rvs(10)

array([7, 6, 5, 4, 8, 4, 5, 5, 0, 1])

In [38]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([107, 103,  75,  87, 114, 106, 101,  99, 105, 103]))

In [39]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.95712558, 0.95636048, 0.4406807 , 0.82250232, 0.75499527,
       0.75727956, 0.19566117, 0.52756242, 0.52128849, 0.72410768])

In [33]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [34]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x15cb20df0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x15cdabd00>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x15ce41070>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x15cdabdf0>},
                   random_state=42)

In [35]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [36]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [37]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86
